In [43]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w
import ml_window as m
from icecream import ic

import itertools
import pickle
import json 

## create variations of data 

In [44]:
# import data
a00, a01 = h.import_desired_data("A", "15T")
window_sched = np.array(a01["Window Open"]) # truth 

In [45]:
arrs = {}
# measured data 
arrs["meas_temp"] = a01["Temp C"]
arrs["meas_rh"] = a01["RH %"]

# ambient data 
arrs["amb_temp"] = a01["Ambient Temp"]
arrs["amb_rh"] = a01["Ambient RH"]

arrs = {k: np.array(h.normalize(v)) for k,v in arrs.items()}

# derivatives
derivs = {f"dt_{k}": h.normalize(np.gradient(v)) for k,v in arrs.items()}

In [46]:
# meausred/ambient temp difference 
diffs = {}
diffs["amb_minus_meas_temp"] = arrs["amb_temp"] - arrs["meas_temp"]
diffs["amb_minus_meas_rh"] = arrs["amb_rh"] - arrs["meas_rh"]

diffs["meas_minus_deriv_temp"] = arrs["meas_temp"] - derivs["dt_meas_temp"]
diffs["meas_minus_deriv_rh"] = arrs["meas_rh"] - derivs["dt_meas_rh"]

diffs = {k: h.normalize(v) for k,v in diffs.items()}


In [47]:
# all data 
all_data = arrs | derivs | diffs
all_data.keys()

dict_keys(['meas_temp', 'meas_rh', 'amb_temp', 'amb_rh', 'dt_meas_temp', 'dt_meas_rh', 'dt_amb_temp', 'dt_amb_rh', 'amb_minus_meas_temp', 'amb_minus_meas_rh', 'meas_minus_deriv_temp', 'meas_minus_deriv_rh'])

In [48]:
# making keys

keys = {}
sets = {}
dict_names= ["arrs", "derivs", "diffs"]
dicts = [arrs, derivs, diffs]

for name, dict_i in zip(dict_names, dicts):
    keys[name] = list(dict_i.keys())
    sets[name] = list(dict_i.keys())



In [49]:
# within set combinations, from length 2, up to the length of number of keys in each dictionary
for k, v in keys.items():
    # print(f"\n  {k}, {len(v)}")
    for i in range(len(v)+1)[2:]:
        for comb in itertools.combinations(v, i):
            sets[k].append(comb)

# across set combinations, only up to length 4 (which excludes a lot of combos but oh well )
keys["combined"] = keys["arrs"] + keys["derivs"] # + keys["diffs"]

sets["combined"] = []
for comb in itertools.combinations(keys["combined"], 4):
        sets["combined"].append(comb)

In [50]:
# find unique elements across all sets
true_sets = {k: set(v) for k,v in sets.items()}
true_set_list = list(true_sets.values())
test_set = list(set.union(*true_set_list))
len(test_set)


113

In [51]:
type(test_set[0])

tuple

In [52]:
test_set = [[i] if type(i) is not tuple else list(i)  for i in test_set]
test_set_data = {
    "data" : test_set
}
test_set_data

{'data': [['amb_minus_meas_temp',
   'meas_minus_deriv_temp',
   'meas_minus_deriv_rh'],
  ['meas_temp', 'meas_rh', 'amb_temp', 'dt_meas_rh'],
  ['dt_meas_rh', 'dt_amb_temp'],
  ['meas_rh', 'amb_temp', 'dt_meas_temp', 'dt_amb_temp'],
  ['dt_meas_temp', 'dt_amb_temp'],
  ['amb_temp', 'dt_meas_temp', 'dt_meas_rh', 'dt_amb_rh'],
  ['meas_temp', 'meas_rh', 'amb_temp', 'dt_meas_temp'],
  ['amb_minus_meas_temp', 'amb_minus_meas_rh', 'meas_minus_deriv_temp'],
  ['meas_temp', 'amb_temp', 'dt_meas_temp', 'dt_meas_rh'],
  ['meas_rh', 'amb_rh', 'dt_meas_temp', 'dt_amb_temp'],
  ['meas_rh', 'dt_meas_rh', 'dt_amb_temp', 'dt_amb_rh'],
  ['meas_temp', 'meas_rh', 'dt_meas_temp', 'dt_meas_rh'],
  ['meas_temp', 'dt_meas_rh', 'dt_amb_temp', 'dt_amb_rh'],
  ['meas_rh', 'dt_meas_temp', 'dt_amb_temp', 'dt_amb_rh'],
  ['meas_rh', 'amb_temp', 'amb_rh'],
  ['meas_temp', 'amb_rh', 'dt_amb_temp', 'dt_amb_rh'],
  ['dt_meas_temp', 'dt_meas_rh', 'dt_amb_temp'],
  ['amb_minus_meas_temp'],
  ['meas_temp', 'amb_rh', '

In [53]:
with open('../constants/test_set_230703.json','w') as f:
    json.dump(test_set_data,f)